<a href="https://colab.research.google.com/github/igoorsa/MachineLearning/blob/main/GradientBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
import pandas as pd
from numpy.random import seed

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [136]:
dados = pd.read_csv('/content/train_titanic.csv')
dados.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [137]:
miss = dados.isnull().sum()/len(dados['Age'])*100
types = dados.dtypes
miss

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [138]:
types

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [139]:
dados.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [140]:
dados.drop(['Cabin','Name','PassengerId','Ticket'],axis = 1, inplace=True)

In [141]:
dados.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [142]:
dados.dropna(axis= 0, inplace =True,subset='Embarked')

In [143]:
types

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [144]:
dados.Sex.value_counts()

male      577
female    312
Name: Sex, dtype: int64

In [145]:
dados.replace('male', 0, inplace = True)
dados.replace('female', 1, inplace = True)

In [146]:
du = pd.get_dummies(dados.Embarked)

In [147]:
concatenados = pd.concat([dados,du], axis = 1)

In [148]:
concatenados.drop('Embarked', axis =1, inplace =True)

In [102]:
x_treino_a = concatenados[concatenados['Age'].notnull()].drop('Age', axis= 1)
x_teste_a = concatenados[concatenados['Age'].isnull()].drop('Age', axis= 1)
y_treino_a = concatenados['Age'][concatenados['Age'].notnull()]

In [103]:
def verShape(a):
  print(a.shape)

In [113]:
verShape(x_treino_a)
verShape(x_teste_a)
verShape(y_treino_a)

(712, 9)
(177, 9)
(712,)


In [96]:
modeloAge = RandomForestRegressor()
modeloAge.fit(x_treino_a,y_treino_a)

RandomForestRegressor()

In [97]:
previsaoAge = modeloAge.predict(x_teste_a)

In [106]:
concatenados['Age'][concatenados['Age'].isnull()] = previsaoAge

<ipython-input-106-8d0b7108eea2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenados['Age'][concatenados['Age'].isnull()] = previsaoAge


In [107]:
concatenados.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
C           0
Q           0
S           0
dtype: int64

In [150]:
x = concatenados.drop('Survived', axis =1)
y = concatenados.Survived

In [151]:
modelo = GradientBoostingClassifier()

skfold = StratifiedKFold()
cross_val_score(modelo,x,y, cv=skfold, n_jobs=-1).mean()

0.8256776487018346

In [152]:
learning_rate = [0.009,0.05,0.1]
max_depth = [6,7,8,9,10]
min_samples_leaf = [6,7,8,9,10]

valores = {'learning_rate':learning_rate,'max_depth':max_depth,
           'min_samples_leaf':min_samples_leaf}

In [153]:
grid = GridSearchCV(modelo,valores,cv=3, n_jobs=-1)
score = grid.fit(x,y)

In [154]:
print(score.best_score_)
print(score.best_estimator_.learning_rate)
print(score.best_estimator_.max_depth)
print(score.best_estimator_.min_samples_leaf)

0.8301672885006218
0.009
6
9
